# Сбор данных кинопоиск и внесение в базу данных neo4j (python)

### Импортируем необходимые библиотеки

In [1]:
import requests
import pandas as pd 
from py2neo import Graph, Node, Relationship

Библиотеки, которые будем использовать: 
* requests - библиотека для запроса и получения данных с сайта 
* pandas - библиотека для создания таблицы из спарсинных данных
* py2neo - предназначена для работы с графовой базой данных Neo4j в Python

### Создаем функцию по сбору данных и сохранения в dataframe pandas

In [2]:
def fetch_movie_data(start_FilmId, end_FildId):
    # Задаем нужные url и доступ к API
    url1 = 'https://kinopoiskapiunofficial.tech/api/v2.2/films/' # URL для получения данных о фильме
    url2 = 'https://kinopoiskapiunofficial.tech/api/v1/staff?filmId=' # URL для получения данных об актерах фильма
    headers = {
    'X-API-KEY': '22deee09-c581-4c94-ab8e-2ad3602a5679',
    'Content-Type': 'application/json'
    }
    final_dataframe = pd.DataFrame()
    # Запускаем цикл сбора данных с заданных FilmId
    for i in range(start_FilmId, end_FildId):
        response1 = requests.get(url=f'{url1}{i}', headers=headers)
        response2 = requests.get(url=f'{url2}{i}', headers=headers)
        # Проверяем существует ли такая страница и начинаем преобразовывать данные
        if response1.status_code == 200 and response2.status_code == 200:
            # Собираем данные о фильме
            json_response1 = response1.json()
            data1 = pd.DataFrame([json_response1])
            data1_v = data1[['nameRu', 'year', 'slogan']]
            # Собираем данные об участниках фильма
            json_response2 = response2.json()
            data2 = pd.DataFrame(json_response2)
            df3 = data2[['nameRu', 'professionKey']]
            new_df = df3.groupby('professionKey').agg({'nameRu': list}).reset_index()
            new_df4 = new_df.set_index('professionKey')
            new_df5 = new_df4.transpose()
            new_df6 = new_df5.reset_index(drop=True)
            # Уберем пустые ('') строки, чтобы не создавать лишних пустых узлов в neo4j
            new_df6['ACTOR'] = new_df6['ACTOR'].apply(lambda x: [item for item in x if item != ''])
            new_df6['DIRECTOR'] = new_df6['DIRECTOR'].apply(lambda x: [item for item in x if item != ''])
            new_df6['PRODUCER'] = new_df6['PRODUCER'].apply(lambda x: [item for item in x if item != ''])
            new_df6['WRITER'] = new_df6['WRITER'].apply(lambda x: [item for item in x if item != ''])
            combined_df = pd.concat([data1_v, new_df6], axis=1)
            # Соединяем две таблицы
            final_dataframe = pd.concat([final_dataframe, combined_df], ignore_index=True)
        else:
            print(f"Ошибка получения данных для фильма с ID {i}: {response1.status_code}")
    return final_dataframe

# Задаем нужные id фильмов для сбора информации
start_FilmId = 300
end_FildId = 320
# Сохраняем полученные данные в переменную result_dataframe
result_dataframe = fetch_movie_data(start_FilmId, end_FildId)

Ошибка получения данных для фильма с ID 315: 404


Здесь взяты 19 фильмов с id 300-319, если нужно больше, то ставить необходимо другие рамки. 

***Ошибка вышла с id 315 - так как такого фильма с таким id не существует (ошибка 404)***

### Теперь покажем какая получилась таблица 

In [3]:
result_dataframe

,nameRu,year,slogan,ACTOR,COMPOSER,DESIGN,DIRECTOR,EDITOR,OPERATOR,PRODUCER,WRITER,TRANSLATOR,VOICE_DIRECTOR
0,Лиззи Магуайр,2003,Lizze McGuire goes to Rome.,"[Хилари Дафф, Адам Лэмберг, Халли Морган, Робе...",[Клифф Эйдельман],"[Дуглас Хиггинс, Патрик Бэнистер, Моника Прудо...",[Джим Фолл],[Маргарет Гудспид],[Ежи Зелиньский],"[Стэн Рогов, Сьюзэн Эстель Джэнсен, Уте Леонха...","[Сьюзэн Эстель Джэнсен, Эд Дектер, Джон Дж. Шт...",NaN,NaN
1,Матрица,1999,Добро пожаловать в реальный мир,"[Киану Ривз, Лоренс Фишбёрн, Кэрри-Энн Мосс, Х...",[Дон Дэвис],"[Оуэн Патерсон, Хью Бэйтап, Мишель МакГэхи, Ки...","[Лана Вачовски, Лилли Вачовски]",[Зак Стэнберг],[Билл Поуп],"[Джоэл Силвер, Брюс Берман, Дэн Краччиоло, Кэр...","[Лилли Вачовски, Лана Вачовски]",[Дмитрий Усачев],[Ярослава Турылёва]
2,Клад,2003,"Кто ищет, тот всегда найдёт","[Сигурни Уивер, Джон Войт, Патриша Аркетт, Шай...",[Джоэл МакНили],"[Мехер Ахмад, Эндрю Макс Кан, Эджи Джерард Род...",[Эндрю Дэвис],"[Томас Дж. Нордберг, Джефри Вулф]",[Стефен Ст. Джон],"[Лоуелл Д. Бланк, Эндрю Дэвис, Майк Медавой, Т...",[Луис Сачар],NaN,[Людмила Демьяненко]
3,Управление гневом,2003,Let the healing begin,"[Адам Сэндлер, Джек Николсон, Мариса Томей, Лу...",[Тедди Кастелуччи],"[Алан Ау, Доменик Сильвестри, Эллен Люттер, Бр...",[Питер Сигал],[Джефф Гурсон],[Дональд МакЭлпайн],"[Бэрри Бернарди, Джек Джиррапуто, Аллегра Клег...",[Дэвид Дорфман],[Ольга Воейкова],[Инна Соболева]
4,Афера,2003,"Это не про деньги, это про ДЕНЬГИ!!!","[Эдвард Бёрнс, Рэйчел Вайс, Моррис Честнат, Ли...",[Кристоф Бек],"[Уильям Арнольд, Мишель Мичел, Мария Нэй]",[Джеймс Фоули],[Стюарт Леви],[Хуан Руис Анчия],"[Майкл Барнс, Марк Бутан, Майкл Оуховен, Майкл...",[Даг Джанг],[Ольга Воейкова],[Наталья Федотова]
5,Телефонная будка,2002,Линия жизни на линии огня,"[Колин Фаррелл, Кифер Сазерленд, Форест Уитаке...",[Гарри Грегсон-Уильямс],"[Эндрю Лоус, Мартин Уист, Дэниэл Орланди, Дон ...",[Джоэл Шумахер],[Марк Стивенс],[Мэттью Либатик],"[Джил Неттер, Дэвид Цукер, Тед Курдила, Элай Р...",[Ларри Коэн],NaN,[Андрей Гриневич]
6,Рекрут,2003,Trust. Betrayal. Deception. In the C.I.A. noth...,"[Аль Пачино, Колин Фаррелл, Бриджет Мойнэхэн, ...",[Клаус Бадельт],"[Эндрю МакАлпайн, Дэннис Девенпорт, Беатрикс А...",[Роджер Дональдсон],[Дэвид Розенблюм],[Стюарт Драйбёрг],"[Джефф Эппл, Гари Барбер, Роджер Бирнбаум, Джо...","[Роджер Таун, Курт Уиммер, Митч Глейзер]",[Ольга Воейкова],[Наталья Федотова]
7,Разыскиваются в Малибу,2003,Ever feel like you don't belong?,"[Джейми Кеннеди, Тэй Диггз, Энтони Андерсон, Р...",[Джон Ван Тонгерен],"[Уильям А. Эллиотт, Дэниэл А. Ломино, Дебра Ли...",[Джон Уайтселл],[Кара Силверман],[Марк Ирвин],"[Факс Бар, Майк Карц, Адам Смолл, Джош Х. Этти...","[Факс Бар, Адам Смолл, Джейми Кеннеди, Ник Сва...",NaN,NaN
8,Тупой и еще тупее тупого: Когда Гарри встретил...,2003,Тупой с первого взгляда,"[Эрик Кристиан Олсен, Дерек Ричардсон, Шери От...",[Эбан Шлеттер],"[Пол Хаггинс, Эндрю Кроуфорд, Кен Табер, Сусан...",[Трой Миллер],[Лоуренс Джордан],[Энтони Б. Ричмонд],"[Орен Каулз, Брэд Кревой, Трой Миллер, Чарльз ...","[Дэнни Байерс, Питер Фаррелли, Беннетт Йеллин,...",NaN,[Леонид Белозорович]
9,Эквилибриум,2002,"Мир, где эмоции караются смертью","[Кристиан Бэйл, Тэй Диггз, Энгус Макфадьен, Шо...",[Клаус Бадельт],"[Вульф Крёгер, Эрик Олсон, Джастин Уорбёртон-Б...",[Курт Уиммер],"[Том Рольф, Уильям Йе]",[Дион Биби],"[Сью Баден-Пауэлл, Ян де Бонт, Лукас Фостер, Э...",[Курт Уиммер],NaN,[Леонид Белозорович]


### Теперь подключаемся к базе данных neo4j и заносим необходимую информацию

In [4]:
# Подключение к базе данных Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

# Создаем словарь действия в фильме каждого участника
profession_types = {
    'ACTOR' : 'ACTED_IN',
    'DIRECTOR' : 'DIRECTED',
    'PRODUCER' : 'PRODUCED',
    'WRITER' : 'WROTE'
}

# Запускаем цикл создания узлов фильмов и участников в neo4j
for index, record in result_dataframe.iterrows():
    movie_title = record['nameRu']
    # Создание узла для фильма
    movie_node = Node("Movie", title=movie_title, year=record['year'], slogan=record['slogan'])
    graph.merge(movie_node, "Movie", "title")
    # Создание узлов для участников
    for profession in ['ACTOR', 'DIRECTOR', 'PRODUCER', 'WRITER']:
        if profession in record:
            for person_name in record[profession]:
                person_node = Node('Person', name=person_name)
                graph.merge(person_node, 'Person', 'name')
                relationship_type = Relationship(person_node, profession_types[profession], movie_node)
                graph.merge(relationship_type) 